# Приложение 1. Код для проектной работы. Сбор данных. Евгений Уланов, АДГО

## Парсинг данных из ВК

Необходимые библиотеки

In [1]:
import re
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

Подгружаем таблицу с вручную собранными группами

In [30]:
groups = pd.read_csv('vk_groups - Sheet1.csv', delimiter=',')

['dybenko_pu35', 'pdshl6', 'lic3cheb', 'podslushano_15_kyzyl', 'podslushano130lyceum', 'podslushka_3lyceum', 'podslyshano127', 'vsev.overheard', 'echoschool22', 'irkschool39', 'bgpkpodslushano', 'school_342', 'shkola_3_zverinets', 'vsevolojsk.school4', 'school_6and4', '7school.veryverycoooool', '16schoollol', 'school242023', 'school26_pds', 'podslushano_shk26', 'school26okk', 'school228037', 'school31_mytischi', 'podslushano33ishi', 'podslushka33_ykt', 'podslushano_school33_podolsk', 'podslushano35nino', 'hot_36', 'podslyshka38', 'chelyabinsk41', 'sharagaa43', 'shigapornoanal', 'publicschool46', 'gdz_ot_putina47', 'mangetka283', 'overheard_school_51', 'podsluhano53', 'vr8682691', 'irk66school', 'ps_67', 'pod_nlo69', 'schoolnomber69', 'school72ul', 'podslushannno__75', 'krasnodar_school93', 'omsk_99', 'schl100', 'samara_102', 'demasluh', 'tsss105', 'mbousoh113', 'school_114.samara', 'podslyshano127', 'podslushano129', 'podslushano_shk131', 'podslushano141', 'podslushano_154']


Функция для сбора постов. count = 100 это максимальное количество постов, которое возможно собрать за один запрос (правило API VK). Но параметр offset помогает делать сразу несколько запросов по 100 постов. 

In [ ]:
def take_posts(groups_df):
    token = "" # мой API ключ
    version = "5.199" # версия API (указана в документации)
    count = 100
    all_posts_data = []

    for index, row in groups_df.iterrows():
        domain = row['domain_group']
        name = row['name']
        region = row['region']
        
        offset = 0
        while offset <= 500:
            response = requests.get('https://api.vk.com/method/wall.get',
                                    params={
                                        'access_token': token,
                                        'v': version,
                                        'domain': domain,
                                        'count': count,
                                        'offset': offset
                                    })
            data = response.json().get('response', {}).get('items', [])
            for post in data:
                post_data = {
                    'name': name,
                    'region': region,
                    'text': post.get('text', ''),
                    'likes': post['likes']['count'],
                    'reposts': post['reposts']['count'],
                    'date': pd.to_datetime(post['date'], unit='s')  # конвертируем timestamp в datetime
                }
                all_posts_data.append(post_data)
            offset += 100
            time.sleep(0.5)
    return pd.DataFrame(all_posts_data)


posts_df = take_posts(groups)



Некоторые группа не имели домена, а имели id. Там нужно указывать owner_id в параметрах вместо domain, подгрузим такие группы отдельно

In [36]:
groups_id = pd.read_csv('vk_id - Sheet1.csv', delimiter=',')

Функция для id

In [35]:
def take_posts_id(groups_df):
    token = "f38d713bf38d713bf38d713bb2f09a3d08ff38df38d713b96429019bb2b30e25b33498e" # мой API ключ
    version = "5.199" # версия API (указана в документации)
    count = 100
    all_posts_data = []

    for index, row in groups_df.iterrows():
        owner_id = row['owner_id']
        name = row['name']
        region = row['region']
        
        offset = 0
        while offset <= 500:
            response = requests.get('https://api.vk.com/method/wall.get',
                                    params={
                                        'access_token': token,
                                        'v': version,
                                        'domain': owner_id,
                                        'count': count,
                                        'offset': offset
                                    })
            data = response.json().get('response', {}).get('items', [])
            for post in data:
                post_data = {
                    'name': name,
                    'region': region,
                    'text': post.get('text', ''),
                    'likes': post['likes']['count'],
                    'reposts': post['reposts']['count'],
                    'date': pd.to_datetime(post['date'], unit='s')  # конвертируем timestamp в datetime
                }
                all_posts_data.append(post_data)
            offset += 100
            time.sleep(0.5)
    return pd.DataFrame(all_posts_data)


posts_df_new = take_posts_id(groups_id)


In [37]:
# объединяем два датафрейма - один с доменами, другой с id. Они ничем не отличаются
frames = [posts_df, posts_df_new]
df = pd.concat(frames)

In [38]:
# сохраним полученный датафрейм
df.to_csv('project_data.csv', index=False)  